In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# 层和块


## 多层感知机的架构

<center><img src="../img/5_deep_learning_computation/modelConstruction.png" width=60%></center>

- 事实证明，“比单个层大”但“比整个模型小”的组件更有价值

- 由**层组**（groups of layers）的重复模式组成
    - 计算机视觉中广泛流行的ResNet-152架构就有数百层

\begin{definition}\label{def:block}
**块**（module, block）：可以描述单个层、由多个层组成的组件或整个模型本身。
\end{definition}


- 块的优势：通过递归可以将一些块组合成更大的组件

<center>
    <img src="../img/5_deep_learning_computation/blocks.svg" width=100% title="多个层被组合成块，形成更大的模型">
</center>
<p style="text-align:center"><b>多个层被组合成块，形成更大的模型</b></p>

\begin{problem}\label{prob:blockimplementation}
计算机如何实现块？
\end{problem}


- 块由**类**（class）表示
    - 任何子类都必须定义一个将其输入转换为输出的**前向传播函数**
    - 存储任何**必要的参数**（有些块不需要参数）
    - 为了计算梯度，块必须具有**反向传播函数**

- 在定义自己的块时，由于自动微分提供了一些后端实现，因此，只需要考虑:
    - **前向传播函数**
    - **必需的参数**

- 回顾一下多层感知机的代码：
    - 包含一个具有256个单元和ReLU激活函数的全连接隐藏层
    - 然后一个具有10个隐藏单元且不带激活函数的全连接输出层

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)  # 输入，2个样本和20个特征
net(X)

tensor([[-0.1823,  0.0080, -0.0137,  0.1294, -0.1146, -0.1092,  0.0671, -0.1275,
          0.1855, -0.1237],
        [-0.1187, -0.1144,  0.0204,  0.1577, -0.1814, -0.1549,  0.1057, -0.0956,
          0.2331, -0.1015]], grad_fn=<AddmmBackward0>)

- 通过实例化`nn.Sequential`来构建我们的模型，层的执行顺序作为参数传递
- `nn.Sequential`定义了一种特殊的`Module`，维护了一个由`Module`组成的有序列表
- 两个全连接层都是`Linear`类的实例，`Linear`类本身就是`Module`的子类

## 自定义块

- 每个块必须具备的基本功能：
    1. 将输入数据作为其前向传播函数的参数
    1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出
    1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的
    1. 存储和访问前向传播计算所需的参数
    1. 根据需要初始化模型参数


- 从零开始编写一个块
    - 包含一个多层感知机，具有256个隐藏单元的隐藏层和一个10维输出层
- 下面的`MLP`类继承了表示块的类
    - 我们只需要提供我们自己的**构造函数**（Python中的`__init__`函数）和**前向传播函数**

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

```python
import torch.nn.functional as F
F.relu(input)
```
- 对输入的张量**按元素**应用relu函数

In [4]:
net = MLP()   # 实例化
net(X)

tensor([[ 0.0225, -0.2324,  0.1964, -0.0633, -0.0183, -0.2165, -0.0312,  0.1983,
          0.0844,  0.0095],
        [ 0.0370, -0.1302,  0.0604, -0.1831,  0.0357, -0.2166, -0.0843,  0.2079,
          0.0165, -0.0483]], grad_fn=<AddmmBackward0>)

- 块的一个主要优点是它的**多功能性**(versatility)：可以子类化块以创建层（如全连接层的类）、整个模型（如上面的`MLP`类）或具有中等复杂度的各种组件

## 顺序块

- 构建我们自己的`Sequential`类：`MySequential`

- 只需要定义两个关键函数：
    1. 一种将块逐个追加到列表中的函数
    1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。把它保存在'Module'类的成员变量_modules中。
            # _module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

- `_modules`的主要优点：在模块的参数初始化过程中，系统知道在`_modules`字典中查找需要初始化参数的子块

- 使用我们的`MySequential`类重新实现多层感知机

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0472, -0.0643, -0.0488, -0.1984,  0.0831,  0.2754,  0.1678, -0.0066,
          0.0578, -0.0003],
        [ 0.1676, -0.0358,  0.1645, -0.2508,  0.0401,  0.1803,  0.1442, -0.0137,
          0.0419,  0.0599]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

- `PyTorch`内置的`Sequential`类使模型构造变得简单，允许组合新的架构，而不必定义自己的类

- 但，并不是所有的架构都是简单的顺序架构，当需要更强的灵活性时，需要定义自己的块

- 有时我们可能希望合并常数参数

\begin{definition}\label{def:constParam}
**常数参数**（constant parameter）：既不是上一层的结果也不是可更新参数的项。
\end{definition}


例如，需要一个计算函数

$$$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$$

其中$\mathbf{x}$是输入，$\mathbf{w}$是参数，$c$是某个在优化过程中没有更新的指定常量

In [7]:
# 为此，实现一个`FixedHiddenMLP`类，如下所示：

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
#         X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(X@self.rand_weight + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2  # 在𝐿1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0247, grad_fn=<SumBackward0>)

- 可以**混合搭配各种组合块的方法**

In [9]:
# 下面的例子以一些想到的方法嵌套块

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP()) # 注意数据的形状
chimera(X)

tensor(0.0969, grad_fn=<SumBackward0>)